<a href="https://colab.research.google.com/github/kobi-2/ML-Lab-IUT/blob/master/170041013_ML_Lab01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is a very simple tutorial intended for the beginners to understand and implement Simple Linear Regression from the scratch. 



<font color='blue'> Simple Linear Regression </font> is a great first machine learning algorithm to implement as it requires you to estimate properties from your training dataset, but is simple enough for beginners to understand. Linear regression is a prediction method that is more than 200 years old. In this tutorial, you will discover how to implement the simple linear regression algorithm from scratch in Python.

After completing this tutorial you will know:<br>
&#9632; How to estimate statistical quantities from training data.<br>
&#9632; How to estimate linear regression coefficients from data.<br>
&#9632; How to make predictions using linear regression for new data.<br>


Linear regression assumes a **linear or straight line relationship between the input variables (X) and the single output variable (y).** More specifically, that output (y) can be calculated from a linear combination of the input variables (X). When there is a single input variable, the method is referred to as a simple linear regression.

In simple linear regression we can use statistics on the training data to estimate the coefficients required by the model to make predictions on new data.

The line for a simple linear regression model can be written as:

$$ y = b_0 + b_1 * x $$
where $b_0$ and $b_1$ are the coefficients we must estimate from the training data. Once the coefficients are known, we can use this equation to estimate output values for $y$ given new input examples of $x$. It requires that you calculate statistical properties from the data such as **mean, variance** and **covariance.**



## <font color = 'blue'> Swedish Insurance Dataset</font>
We will use a real dataset to demonstrate simple linear regression. The dataset is called the **“Auto Insurance in Sweden”** dataset and involves **<font color='blue'> predicting the total payment for all the claims in thousands of Swedish Kronor (y) given the total number of claims (x). </font>**

This means that for a new number of claims (x) we will be able to predict the total payment of claims (y).

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Let's load some basic python libraries that we will need over the course of this tutorial. 

In [ ]:
# library for manipulating the csv data
import pandas as pd

# library for scientific calculations on numbers + linear algebra
import numpy as np
import math

# library for regular plot visualizations
import matplotlib.pyplot as plt

#library for responsive visualizations
import plotly.express as px


### Reading the data

Manual Data Input or Google Drive Mount is **NOT REQUIRED**. Data is collected from [this link of google sheet that is published to the web.](https://docs.google.com/spreadsheets/d/e/2PACX-1vT5HJttPQr4IxRmaHDRGaO8IdXaDu_nvcGPZpWfTiqX6UxQ_5FnEx-E-T8FJbea41V0W2bEDSojp3on/pub?gid=1493896807&single=true&output=csv) 


In [ ]:
# data = pd.read_csv('/content/swedish_insurance.csv')
data = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vT5HJttPQr4IxRmaHDRGaO8IdXaDu_nvcGPZpWfTiqX6UxQ_5FnEx-E-T8FJbea41V0W2bEDSojp3on/pub?gid=1493896807&single=true&output=csv')
data.sort_values('X',inplace=True)
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63 entries, 30 to 3
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   X       63 non-null     int64  
 1   Y       63 non-null     float64
dtypes: float64(1), int64(1)
memory usage: 1.5 KB


In [ ]:
print(data.columns)
data.head(10)

Index(['X', 'Y'], dtype='object')


,X,Y
30,0,0.0
15,2,6.6
49,3,39.9
23,3,13.2
18,3,4.4
29,4,38.1
38,4,12.6
26,4,11.8
33,5,40.3
10,5,20.9


Let's have a look at the data itself. You can either use `matplotlib.pyplot` or `plotly` for visualization. The latter one produces responsive visualizations. Try hovering over the points on the graph to see the actual values.

In [ ]:
fig = px.box(data['X'], points = 'all')
fig.update_layout(title = f'Distribution of X',title_x=0.5, yaxis_title= "Number of Insurance Claims")
fig.show()

fig = px.box(data['Y'], points = 'all')
fig.update_layout(title = f'Distribution of Y',title_x=0.5, yaxis_title= "Amount of Insurance Paid")
fig.show()

In [ ]:
fig = px.scatter(x = data['X'], y=data['Y'])
fig.update_layout(title = 'Swedish Automobiles Data', title_x=0.5, xaxis_title= "Number of Claims", yaxis_title="Payment in Claims", height = 500, width = 700)
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.show()

**This tutorial is broken down into five parts:<br>**
&#9832; Calculate Mean and Variance.<br>
&#9832; Calculate Covariance (X,Y).<br>
&#9832; Estimate Coefficients.<br>
&#9832; Make Predictions.<br>
&#9832; Visual Comparison for Correctness.<br>
These steps will give you the foundation you need to implement and train simple linear regression models for your own prediction problems.

### 1. Calculate Mean and Variance.
As said earlier, simple linear regression uses mean and variance of the given data. We will use `numpy` builtin functions to calculate them. 

In [ ]:
data['Y']

30      0.0
15      6.6
49     39.9
23     13.2
18      4.4
      ...  
5     170.9
41    202.4
36    217.6
0     392.5
3     422.2
Name: Y, Length: 63, dtype: float64

In [ ]:
mean_x = np.mean(data['X'])
mean_y = np.mean(data['Y'])

var_x = np.var(data['X'])
var_y = np.var(data['Y'])


print('x stats: mean= %.3f   variance= %.3f' % (mean_x, var_x))
print('y stats: mean= %.3f   variance= %.3f' % (mean_y, var_y))

x stats: mean= 22.905   variance= 536.658
y stats: mean= 98.187   variance= 7505.052


### 2. Calculate Covariance.
The covariance of two groups of numbers describes how those numbers change together. Covariance is a generalization of correlation. Correlation describes the relationship between two groups of numbers, whereas covariance can describe the relationship between two or more groups of numbers. It is calculated by the following formula. 
$$ Cov(X,Y) = \frac{\sum{(X_i - \overline{X})}{(Y_j - \overline{Y})}}{n} $$

You can simply implement it by yourself or use builtin function `numpy.cov()`


In [ ]:
# Calculate covariance between x and y
def covariance(x, y):
    mean_x = np.mean(x)
    mean_y = np.mean(y)
    covar = 0.0
    for i in range(len(x)):
        covar += (x[i] - mean_x) * (y[i] - mean_y)
    return covar/len(x)



covar_xy = covariance(data['X'], data['Y'])
print(f'Cov(X,Y): {covar_xy}')

Cov(X,Y): 1832.0543461829186


### 3. Estimate Coefficients
We must estimate the values for two coefficients in simple linear regression.

In [ ]:
b1 = covar_xy / var_x
b0 = mean_y - b1 * mean_x

print(f'Coefficents:\n b0: {b0}  b1: {b1} ')


Coefficents:
 b0: 19.994485759114767  b1: 3.413823560066369 


### 4. Make Predictions
The simple linear regression model is a line defined by coefficients estimated from training data. Once the coefficients are estimated, we can use them to make predictions. The equation to make predictions with a simple linear regression model is as follows:
$$ \hat{y} = b_0 + b_1 * x $$

In [ ]:
x = data['X'].values.copy()
x

array([  0,   2,   3,   3,   3,   4,   4,   4,   5,   5,   6,   6,   6,
         7,   7,   7,   8,   8,   9,   9,   9,  10,  11,  11,  11,  12,
        13,  13,  13,  13,  14,  14,  15,  16,  17,  19,  20,  22,  23,
        23,  23,  24,  24,  25,  26,  27,  29,  29,  30,  31,  37,  40,
        41,  41,  45,  48,  53,  55,  57,  60,  61, 108, 124])

In [ ]:
# Taking the values from the dataframe 
x = data['X'].values.copy()
print(f'x: {x}')

# Predicting the new data based on calculated coeffiecents. 
y_hat = b0 + b1 * x
print(f'\n\ny_hat: {y_hat}')

y = data['Y'].values
print(f'\n\ny: {y}')

x: [  0   2   3   3   3   4   4   4   5   5   6   6   6   7   7   7   8   8
   9   9   9  10  11  11  11  12  13  13  13  13  14  14  15  16  17  19
  20  22  23  23  23  24  24  25  26  27  29  29  30  31  37  40  41  41
  45  48  53  55  57  60  61 108 124]


y_hat: [ 19.99448576  26.82213288  30.23595644  30.23595644  30.23595644
  33.64978     33.64978     33.64978     37.06360356  37.06360356
  40.47742712  40.47742712  40.47742712  43.89125068  43.89125068
  43.89125068  47.30507424  47.30507424  50.7188978   50.7188978
  50.7188978   54.13272136  57.54654492  57.54654492  57.54654492
  60.96036848  64.37419204  64.37419204  64.37419204  64.37419204
  67.7880156   67.7880156   71.20183916  74.61566272  78.02948628
  84.8571334   88.27095696  95.09860408  98.51242764  98.51242764
  98.51242764 101.9262512  101.9262512  105.34007476 108.75389832
 112.16772188 118.995369   118.995369   122.40919256 125.82301612
 146.30595748 156.54742816 159.96125172 159.96125172 173.61654596
 183.8

### 5. Visual Comparison for Correctness 

In [ ]:
import plotly.graph_objects as go
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['X'], y=data['Y'], name='train', mode='markers', marker_color='rgba(152, 0, 0, .8)'))
fig.add_trace(go.Scatter(x=data['X'], y=y_hat, name='prediction', mode='lines+markers', marker_color='rgba(0, 152, 0, .8)'))

fig.update_layout(title = f'Swedish Automobiles Data\n (visual comparison for correctness)',title_x=0.5, xaxis_title= "Number of Claims", yaxis_title="Payment in Claims")
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.show()

## Where To Go From Here 
* <font color="red">Can you find out the **mean squared error (MSE)** of the predictions???</font>
* Extend the same problem for multiple input features. 



---



# Finding MSE (Mean Squared Error): 



In [ ]:
data_y = data['Y'].values.copy()
e = np.subtract(data_y, y_hat)
sqe = e**2
sumsqe = np.sum(sqe)
mse = sumsqe/len(data_y)
print('MSE: ', mse)

MSE:  1250.7419293814687


<br>

   ---

<br>

#**Report:**

**1.**  From the data we try to fit a simple linear regression line and try to predict the all Claimed Payment amount in thousands of Swedish Kronor (y) given the total number of claims (x).

<br>

**2.** We cannot fully, without any error predict the outcome. But the line gives us a good prediction against the actual outcome. 

  However, to have a more solid prediction, larger amount of data should be used. The more amount of data, the better the model.

<br>

**3.** Here, we have used MSE as a loss function. For the use of any other loss function we might yield a different outcome.

<br>

**4.** The regression line is heavily sensitive to the data, specially for the outliers. If we look at the data, we will see that there are two points on the far top right corner which act as outlier here. If you change the values, we will see that these affect the regression line greatly. 

This is demonstrated below: 


**(a)** Firstly, We will copy the Y values from the data and change the last 2 values to a very low values of y:

In [ ]:
modified_y = data_y
modified_y[-2] = 150.5
modified_y[-1] = 170.3
modified_y

array([  0. ,   6.6,  39.9,  13.2,   4.4,  38.1,  12.6,  11.8,  40.3,
        20.9,  50.9,  14.6,  14.8,  27.9,  48.8,  77.5,  76.1,  55.6,
        48.7,  87.4,  52.1,  65.3,  57.2,  21.3,  23.5,  58.1,  31.9,
        15.7,  93. ,  89.9,  77.5,  95.5,  32.1,  59.6, 142.1,  46.2,
        98.1, 161.5, 113. ,  39.6,  56.9, 137.9, 134.9,  69.2, 187.5,
        92.6, 103.9, 133.3, 194.5, 209.8, 152.8, 119.4,  73.4, 181.3,
       214. , 248.1, 244.6, 162.8, 170.9, 202.4, 217.6, 150.5, 170.3])

<br>

**(b)** Now we will find the covariance of $x$ and $y_{modified}$, and mean of $y$ and coefficients.

<br> 

Here for the second time our expression will be:
$$ \hat{y}_{modified} = b_3 + b_4 * x $$

<br>

In [ ]:
covar_xy_2 = covariance(data['X'], modified_y)
print(f'Cov(X,Y) with modified y: {covar_xy_2}')

mean_y_2 = np.mean(modified_y)

b1_modified = covar_xy_2 / var_x
b0_modified = mean_y_2 - b1_modified * mean_x

print(f'\nCoefficents:\n b0_modified : {b0_modified}  b1_modified: {b1_modified} ')


Cov(X,Y) with modified y: -123.37959183673459

Coefficents:
 b0_modified : 95.61350961552006  b1_modified: -0.22990374620773565 


<br>

**(c)** Now we predict for the modified Y data:

<br>


In [ ]:
print(f'x: {x}')

# Predicting the new data based on calculated coeffiecents 
y_hat_2 = b0_modified + b1_modified * x
print(f'\n\n y_hat_2: {y_hat_2}')

print('\n\n modified y: last 2 values: ', modified_y[-2:])

y = data['Y'].values
print('\n\n actual y: last 2 values: ', y[-2:])

x: [  0   2   3   3   3   4   4   4   5   5   6   6   6   7   7   7   8   8
   9   9   9  10  11  11  11  12  13  13  13  13  14  14  15  16  17  19
  20  22  23  23  23  24  24  25  26  27  29  29  30  31  37  40  41  41
  45  48  53  55  57  60  61 108 124]


 y_hat_2: [95.61350962 95.15370212 94.92379838 94.92379838 94.92379838 94.69389463
 94.69389463 94.69389463 94.46399088 94.46399088 94.23408714 94.23408714
 94.23408714 94.00418339 94.00418339 94.00418339 93.77427965 93.77427965
 93.5443759  93.5443759  93.5443759  93.31447215 93.08456841 93.08456841
 93.08456841 92.85466466 92.62476091 92.62476091 92.62476091 92.62476091
 92.39485717 92.39485717 92.16495342 91.93504968 91.70514593 91.24533844
 91.01543469 90.5556272  90.32572345 90.32572345 90.32572345 90.09581971
 90.09581971 89.86591596 89.63601221 89.40610847 88.94630098 88.94630098
 88.71639723 88.48649348 87.10707101 86.41735977 86.18745602 86.18745602
 85.26784104 84.5781298  83.42861107 82.96880357 82.50899608 81.8192848

<br>

**(d)** Now if we plot the new regression model against the new modified Y data: 

<br>


In [ ]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data['X'], y=modified_y, name='train 2', mode='markers', marker_color='rgba(152, 0, 152, .8)'))
fig.add_trace(go.Scatter(x=data['X'], y=y_hat_2, name='prediction 2', mode='lines+markers', marker_color='rgba(0, 152, 0, .8)'))

fig.update_layout(title = f'Effect of Outliers of Y data',title_x=0.5, xaxis_title= "Number of Claims", yaxis_title="Payment in Claims (with modified y data)")
fig.update_xaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.update_yaxes(showline=True, linewidth=2, linecolor='black', mirror=True)
fig.show()

<br>

**(e)** And the new MSE will be: 

<br>


In [ ]:
e_2 = np.subtract(modified_y, y_hat_2)
sqe_2 = e_2**2
sumsqe_2 = np.sum(sqe_2)
mse_2 = sumsqe_2/len(modified_y)
print('MSE of new modified Y data: ', mse_2)

MSE of new modified Y data:  5062.832226430148


<br>

Conclusion: <br>

As we can see, for the outlier two points in the actual dataform $(x,y) = (108, 392.5) $ and $ (124, 422.2) $, if we changed them to a newer and lower value of y of $(x,y) = (108, 150.5) $ and $ (124, 170.3) $ , then the whole prediction changes hugely. The linear regression model changes drastically and MSE (Mean Sqaured Error) increases largely as well. 

This is because of the effect that the outliers have on linear regression model. To solve this, we can include even more data and clean the data from outliers. 